In [3]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import datetime as dt
import pandas_ta as ta
import yfinance as yf
import copy
import random
from pymongo import MongoClient
import sys
from dotenv import load_dotenv
import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

load_dotenv()
mongo_client = MongoClient(os.environ.get("MONGO_URI"))
stock_db = mongo_client["stock_db"]

- Đọc và chuẩn bị dữ liệu

In [4]:
def decode_data(file_path):
    data = np.fromfile(file_path, dtype=np.uint8)
    record_size = 32 
    num_records = len(data) // record_size
    num_columns = record_size // 4 
    raw_data = data.reshape(num_records, record_size // 4, 4)
    int_data = raw_data[:, :2].view(np.int32) 
    float_data = raw_data[:, 2:].view(np.float32)
    records = np.hstack((int_data, float_data))
    records = records.reshape(num_records, num_columns)
    records = records[::-1]
    df = pd.DataFrame(records, columns=[f"Col_{i}" for i in range(num_columns)])
    return df

def get_file_name_list(folder_path):
    file_name_list = []
    files = os.listdir(folder_path)
    for file in files:
        file_name_list.append(file[:-4])
    return file_name_list

def clean_eod_stock_data(df_raw):
    df_raw = df_raw[df_raw['Col_0'] > 20200000]
    df_raw['cap'] = (df_raw['Col_5'] * df_raw['Col_7'])/1000000
    df_clean = df_raw.drop(columns=['Col_1', 'Col_7'])
    df_clean['Col_0'] = pd.to_datetime(df_clean['Col_0'], format='%Y%m%d')
    df_clean.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'cap']

    return df_clean.reset_index(drop=True)

eod_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\stock"

In [5]:
full_stock_dict = {}
for stock in [stock for stock in get_file_name_list(eod_stock_folder_path) if len(stock) == 3]:
    temp_file_path = eod_stock_folder_path + f'\\{stock}.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_eod_stock_data(temp_df_raw)
    temp_df_clean.insert(0, 'stock', stock)
    full_stock_dict[stock] = temp_df_clean

for item, df in full_stock_dict.items():
    full_stock_dict[item]['ma20_volume'] = full_stock_dict[item]['volume'][::-1].rolling(window=20, min_periods=1).mean()[::-1]
    full_stock_dict[item]['value'] = full_stock_dict[item]['volume']*full_stock_dict[item]['close']
    full_stock_dict[item]['ma20_value'] = full_stock_dict[item]['value'][::-1].rolling(window=20, min_periods=1).mean()[::-1]

- Các hàm tính toán

In [6]:
# Hàm để tính ngày bắt đầu và kết thúc của một quý
def get_quarter_dates(year, quarter):
    if quarter == "q1":
        start_date = f"{year-1}-10-01"
        end_date = f"{year-1}-12-31"
    elif quarter == "q2":
        start_date = f"{year}-01-01"
        end_date = f"{year}-03-31"
    elif quarter == "q3":
        start_date = f"{year}-04-01"
        end_date = f"{year}-06-30"
    elif quarter == "q4":
        start_date = f"{year}-07-01"
        end_date = f"{year}-09-30"
    return start_date, end_date

# Hàm để xác định quý hiện tại
def get_current_quarter_year(name):
    now = datetime.now()
    year = now.year
    month = now.month
    if 1 <= month <= 3:
        quarter = "q1"
    elif 4 <= month <= 6:
        quarter = "q2"
    elif 7 <= month <= 9:
        quarter = "q3"
    else:
        quarter = "q4"
    if name == 'quarter':
        return quarter
    elif name == 'year':
        return year

- Tạo dict chứa các mốc thời gian từng quý

In [7]:
# Lấy ra quý và năm hiện tại
current_quarter = get_current_quarter_year('quarter')
current_year = int(get_current_quarter_year('year'))

# Tạo danh sách các quý và năm
quarters = ["q1", "q2", "q3", "q4"]
years = range(2020, current_year + 1)

# Tạo dict chứa các quý và mốc thời gian
quarter_date_dict = {}
for year in years:
    for quarter in quarters:
        if year == 2020: #Check điều kiện để không thêm quý 1 năm 2020
            if quarter != 'q1': 
                quarter_date_dict[f"{quarter}_{year}"] = get_quarter_dates(year, quarter)
        elif year == current_year: #Check điều kiện để không thêm các quý lớn hơn quý hiện tại
            if quarter <= current_quarter:
                quarter_date_dict[f"{quarter}_{year}"] = get_quarter_dates(year, quarter)
        else:
            quarter_date_dict[f"{quarter}_{year}"] = get_quarter_dates(year, quarter)

- Chuẩn bị danh sách cổ phiếu từng quý

In [8]:
#Tạo dict chứa danh cách cổ phiếu từng quý
quarter_stock_list_dict = {}
for quarter, date in quarter_date_dict.items():
    temp_dict = {k: v[(v['date'] >= date[0]) & (v['date'] <= date[1])] for k, v in copy.deepcopy(full_stock_dict).items()}
    temp_dict = [k for k, v in temp_dict.items() if (v['volume'].mean() >= 50000) & (v['ma20_volume'].min() >= 20000)&
                                                    (v['value'].mean() >= 500000) & (v['ma20_value'].min() >= 200000)]
    quarter_stock_list_dict[quarter] = temp_dict

#Tạo danh sách tất cả cổ phiếu quý trước
previous_full_stock_list = set()
for quarter in list(quarter_stock_list_dict.keys())[:-1]:
    stock_list = quarter_stock_list_dict[quarter]
    previous_full_stock_list.update(stock_list)

#Tạo danh tất cả sách cổ phiếu quý này
full_stock_list = set()
for quarter, stock_list in quarter_stock_list_dict.items():
    full_stock_list.update(stock_list)

#Thêm danh sách các cổ phiếu vào vị trí đầu tiên của dict
quarter_stock_list_dict = {**{'all_stock': list(full_stock_list)}, **quarter_stock_list_dict}

#In ra số lượng các cổ phiếu
for key, value in quarter_stock_list_dict.items():
    print(key, len(value))

#In ra các cổ phiếu mới làn đầu xuất hiện
new_stock_list = full_stock_list - previous_full_stock_list
print(f'Các cổ phiếu mới có: {new_stock_list}')
new_stock_df = pd.DataFrame(new_stock_list, columns=['new_stock_list'])

# Điền thêm giá trị NaN để đảm bảo các danh sách có cùng chiều dài
data = copy.deepcopy(quarter_stock_list_dict)
for key in data:
    if len(data[key]) < len(full_stock_list):
        data[key].extend([np.nan] * (len(full_stock_list) - len(data[key])))

# Tạo DataFrame
quarter_stock_list = pd.DataFrame(data)

all_stock 645
q2_2020 194
q3_2020 244
q4_2020 270
q1_2021 303
q2_2021 392
q3_2021 416
q4_2021 424
q1_2022 529
q2_2022 508
q3_2022 444
q4_2022 393
q1_2023 353
q2_2023 310
q3_2023 351
q4_2023 406
q1_2024 342
q2_2024 366
q3_2024 393
q4_2024 360
q1_2025 332
Các cổ phiếu mới có: {'HVA', 'MZG', 'TRC', 'RYG'}


- Lưu lại dữ liệu

In [11]:
# Hàm để tính ngày bắt đầu và kết thúc của một quý
def get_real_quarter_dates(year, quarter):
    if quarter == "q1":
        start_date = f"{year}-01-01"
        end_date = f"{year}-03-31"
    elif quarter == "q2":
        start_date = f"{year}-04-01"
        end_date = f"{year}-06-30"
    elif quarter == "q3":
        start_date = f"{year}-07-01"
        end_date = f"{year}-09-30"
    elif quarter == "q4":
        start_date = f"{year}-10-01"
        end_date = f"{year}-12-31"
    return [start_date, end_date]

# Lấy ra quý và năm hiện tại
current_quarter = get_current_quarter_year('quarter')
current_year = int(get_current_quarter_year('year'))

# Tạo danh sách các quý và năm
quarters = ["q1", "q2", "q3", "q4"]
years = range(2020, current_year + 1)

# Tạo dict chứa các quý và mốc thời gian
real_quarter_date_dict = {}
for year in years:
    for quarter in quarters:
        if year == 2020: #Check điều kiện để không thêm quý 1 năm 2020
            if quarter != 'q1': 
                real_quarter_date_dict[f"{quarter}_{year}"] = get_real_quarter_dates(year, quarter)
        elif year == current_year: #Check điều kiện để không thêm các quý lớn hơn quý hiện tại
            if quarter <= current_quarter:
                real_quarter_date_dict[f"{quarter}_{year}"] = get_real_quarter_dates(year, quarter)
        else:
            real_quarter_date_dict[f"{quarter}_{year}"] = get_real_quarter_dates(year, quarter)

for quarter, date in real_quarter_date_dict.items():
    date.append(len(quarter_stock_list_dict[quarter]))

quarter_map = pd.DataFrame.from_dict(real_quarter_date_dict, orient='index').reset_index()
quarter_map.columns = ['quarter', 'start_date', 'end_date', 'stock_count']

In [12]:
with pd.ExcelWriter('../.xlsx/quarter_stock_map.xlsx', engine='openpyxl') as writer:
    quarter_stock_list.to_excel(writer, sheet_name='quarter_stock_list', index=False)
    quarter_map.to_excel(writer, sheet_name='quarter_map', index=False)
    new_stock_df.to_excel(writer, sheet_name='new_stock_df', index=False)

In [ ]:
# from sqlalchemy import create_engine, text

# def safe_to_sql(df, table_name, engine, schema=None, verbose=True):
#     """
#     Lưu DataFrame vào SQL an toàn với thời gian gián đoạn tối thiểu sử dụng chiến lược đổi tên.
    
#     Args:
#         df: pandas DataFrame cần lưu
#         table_name: tên bảng đích
#         engine: SQLAlchemy engine 
#         schema: tên schema database (tùy chọn)
#         verbose: in thông báo tiến trình (mặc định True)
    
#     Returns:
#         bool: True nếu thành công
#     """
    
#     # Định nghĩa tên bảng
#     temp_table_name = f"temp_{table_name}" 
#     old_table_name = f"old_{table_name}"
    
#     # Tạo tên bảng đầy đủ với schema
#     schema_prefix = f"{schema}." if schema else ""
#     full_table_name = f"{schema_prefix}{table_name}"
#     full_temp_table_name = f"{schema_prefix}{temp_table_name}"
#     full_old_table_name = f"{schema_prefix}{old_table_name}"
    
#     try:
#         # Xóa bảng tạm nếu đã tồn tại từ lần thử trước đó
#         with engine.begin() as conn:
#             conn.execute(text(f"DROP TABLE IF EXISTS {full_temp_table_name}"))
        
#         # 1. Ghi dữ liệu vào bảng tạm
#         df.to_sql(temp_table_name, engine, schema=schema, if_exists='replace', index=False)
        
#         if verbose:
#             print(f"Đã tạo bảng tạm {full_temp_table_name}")
        
#         # 2. Tạo giao dịch cho các thao tác đổi tên
#         with engine.begin() as conn:
#             # Kiểm tra xem bảng gốc có tồn tại không
#             exists_query = f"SELECT to_regclass('{full_table_name}')"
#             exists = conn.execute(text(exists_query)).scalar() is not None
            
#             if exists:
#                 # 3. Đổi tên bảng gốc thành old_
#                 conn.execute(text(f"ALTER TABLE {full_table_name} RENAME TO {old_table_name}"))
#                 if verbose:
#                     pass
#                     # print(f"Đã đổi tên bảng gốc thành {full_old_table_name}")
            
#             # 4. Đổi tên bảng tạm thành tên bảng gốc
#             conn.execute(text(f"ALTER TABLE {full_temp_table_name} RENAME TO {table_name}"))
#             if verbose:
#                 pass
#                 # print(f"Đã đổi tên bảng tạm thành {full_table_name}")
            
#             if exists:
#                 # 5. Xóa bảng cũ
#                 conn.execute(text(f"DROP TABLE IF EXISTS {full_old_table_name}"))
#                 if verbose:
#                     pass
#                     # print(f"Đã xóa bảng cũ {full_old_table_name}")
        
#         if verbose:
#             print(f"Đã lưu dữ liệu thành công vào bảng {full_table_name}")
#         return True
        
#     except Exception as e:
#         if verbose:
#             print(f"Lỗi khi lưu vào SQL: {str(e)}")
#         # Cố gắng dọn dẹp nếu có thể
#         try:
#             with engine.begin() as conn:
#                 conn.execute(text(f"DROP TABLE IF EXISTS {full_temp_table_name}"))
#         except:
#             pass
#         return False
    
# host = '14.225.198.55'
# port = '5432'
# dbname = 'stock_db'
# user = 'admin'
# password = 'Chodom!123'

# connection_url = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
# engine = create_engine(connection_url)

# # Tên bảng
# safe_to_sql(history_stock_df, "history_stock", engine, schema=None, verbose=True)
# safe_to_sql(history_index_df, "history_index", engine, schema=None, verbose=True)
# safe_to_sql(history_ms_chart_df, "history_ms_chart", engine, schema=None, verbose=True)
# safe_to_sql(history_group_df, "history_group", engine, schema=None, verbose=True)
# safe_to_sql(history_portfolio_score_df, "history_portfolio_score", engine, schema=None, verbose=True)